In [ ]:
import pandas as pd

depth_and_num_params_csv_path = 'depth_and_num_params.csv'
evaluate_csv_path = 'evaluate.csv'
problem_scale = 4

# 设置显示选项
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 输入路径
depth_and_num_params_csv_path = 'depth_and_num_params.csv'
evaluate_csv_path = 'evaluate.csv'
problem_scale = 4

# 读入 Depth 和 Params 数据
df = pd.read_csv(depth_and_num_params_csv_path)
df.loc[df["method"] == "HeaSolver", "method"] = "HEA"
df.loc[df["method"] == "PenaltySolver", "method"] = "P-QAOA"
df.loc[df["method"] == "ChocoSolver", "method"] = "Choco-Q"
df.loc[df["method"] == "RasenganSolver", "method"] = "Rasengan"

benchmarks = ["F", "K", "J", "S", "G"]
method_order = ['HEA', 'P-QAOA', 'Choco-Q', 'Rasengan']

# pkid → Benchmark label，如 F1, F2, ..., G4
pkid_to_label = {
    pkid: f"{b}{i}"
    for pkid, (b, i) in enumerate(
        ((b, i) for b in benchmarks for i in range(1, problem_scale + 1))
    )
}
df["Benchmark"] = df["pkid"].map(pkid_to_label)
df["method"] = pd.Categorical(df["method"], categories=method_order, ordered=True)

# 聚合
df_grouped = df.groupby(["method", "Benchmark"], observed=True).mean(numeric_only=True).reset_index()

# 构建 pivot 表格
pivot_depth = df_grouped.pivot(index="method", columns="Benchmark", values="depth")
pivot_param = df_grouped.pivot(index="method", columns="Benchmark", values="num_params")
pivot_depth.loc["Rasengan"] = pivot_depth.loc["Rasengan"] / pivot_param.loc["Rasengan"]

# ARG 数据处理
df_eval = pd.read_csv(evaluate_csv_path)
df_eval.loc[df_eval["method"] == "HeaSolver", "method"] = "HEA"
df_eval.loc[df_eval["method"] == "PenaltySolver", "method"] = "P-QAOA"
df_eval.loc[df_eval["method"] == "ChocoSolver", "method"] = "Choco-Q"
df_eval.loc[df_eval["method"] == "RasenganSolver", "method"] = "Rasengan"
df_eval = df_eval[df_eval['ARG'] <= 100000]
df_eval["Benchmark"] = df_eval["pkid"].map(pkid_to_label)
df_eval["method"] = pd.Categorical(df_eval["method"], categories=method_order, ordered=True)
df_arg_grouped = df_eval.groupby(["method", "Benchmark"], observed=True)["ARG"].mean().reset_index()
pivot_arg = df_arg_grouped.pivot(index="method", columns="Benchmark", values="ARG")

# 列顺序对齐
column_order = list(pkid_to_label.values())
pivot_depth = pivot_depth[column_order]
pivot_param = pivot_param[column_order]
pivot_arg = pivot_arg[column_order]

# 四舍五入

# 多级索引
depth_labeled = pivot_depth.copy()
depth_labeled.index = pd.MultiIndex.from_product([["Depth"], depth_labeled.index])
param_labeled = pivot_param.copy()
param_labeled.index = pd.MultiIndex.from_product([["#Params"], param_labeled.index])
arg_labeled = pivot_arg.copy()
arg_labeled.index = pd.MultiIndex.from_product([["ARG"], arg_labeled.index])

# === 计算 improvement ===
def compute_improvement(target_df, baseline="Rasengan"):
    improvements = {}
    for method in method_order:
        if method == baseline:
            improvements[method] = None
            continue
        ratio = target_df.loc[method] / target_df.loc[baseline]
        improvements[method] = ratio.mean()
    return improvements

def append_improvement_column(df, improvements):
    df = df.copy()
    df["improvement"] = [
        round(val, 3) if val is not None else pd.NA
        for val in [improvements.get(method, pd.NA) for method in df.index.get_level_values(1)]
    ]
    return df

# 分别计算 improvement 并添加列
arg_improvements = compute_improvement(pivot_arg)
depth_improvements = compute_improvement(pivot_depth)
param_improvements = compute_improvement(pivot_param)

depth_labeled = depth_labeled.round().astype("Int64").astype(str)
param_labeled = param_labeled.round().astype("Int64").astype(str)
arg_labeled = arg_labeled.round(3)

arg_labeled = append_improvement_column(arg_labeled, arg_improvements)
depth_labeled = append_improvement_column(depth_labeled, depth_improvements)
param_labeled = append_improvement_column(param_labeled, param_improvements)

# 合并最终表格
merged_with_improvement = pd.concat([arg_labeled, depth_labeled, param_labeled])
merged_with_improvement.to_pickle("table_2.pkl")

# 显示结果
merged_with_improvement

Benchmark          F1    F2    F3    F4    G1    G2    G3    G4   J1    J2  \
        method                                                               
Depth   HEA        46    91   121   156    76    91   116   136   51    66   
        P-QAOA     87   148   174   207   260   309   745   785  110   133   
        Choco-Q   507  1888  2688  3848  1878  1678  3207  3657  982  2027   
        Rasengan   34    49    60    58    55    64    38    62   60    72   
#Params HEA        90   225   315   420   180   225   300   360  105   150   
        P-QAOA     10    10    10    10    10    10    10    10   10    10   
        Choco-Q    10    10    10    10    10    10    10    10   10    10   
        Rasengan    3    19    34    66     8     5    20    15    3     6   

Benchmark           J3    J4    K1    K2    K3    K4   S1    S2    S3    S4  \
        method                                                                
Depth   HEA         86   106    56    91   106   121   61    76    96   116   
        P-QAOA     167   190   172   250   287   320  228   278   398   491   
        Choco-Q   3217  4442  1324  3532  4406  5268  981  1687  1836  2104   
        Rasengan    66    72    85    87    87    87   25    26    22    23   
#Params HEA        210   270   120   225   270   315  135   180   240   300   
        P-QAOA      10    10    10    10    10    10   10    10    10    10   
        Choco-Q     10    10    10    10    10    10   10    10    10    10   
        Rasengan    17    26     4     8    21    21   10    15    27    64   

Benchmark        improvement  
        method                
Depth   HEA             1.98  
        P-QAOA         6.693  
        Choco-Q       48.487  
        Rasengan        <NA>  
#Params HEA           18.844  
        P-QAOA         1.093  
        Choco-Q        1.093  
        Rasengan        <NA>